In [92]:
import numpy as np
from scipy import special
from nltk.tokenize import RegexpTokenizer
from nltk import word_tokenize
from stop_words import get_stop_words
#pip install stop-words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,HashingVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [3]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."

# compile sample documents into a list
#doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

doc_set = [
    'weather: warm, cold, freezing, hot, windy,warm',
    'weather: dry, windy, moist, cold, etc',
    'freezing means dry and windy',
    'sports game, be it basketball, hockey or soccer, I feel better',
    'sports can be soothing. hockey, but I like soccer and basketball'
]

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20

dataset = fetch_20newsgroups(shuffle=True, random_state=1,remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]

doc_set = data_samples[:100]

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    #tokens = word_tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)

#print(dictionary.token2id)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

print corpus[0]
print dictionary

[(0, 4), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 4), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 4), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 2), (44, 1), (45, 2), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1)]
Dictionary(4581 unique tokens: [u'repris', u'demand', u'hitch', u'four', u'164']...)


In [260]:
def log_sum(log_a,log_b) :
    if (log_a < log_b) :
        v = log_b+np.log(1 + np.exp(log_a-log_b))
    else :
        v = log_a+np.log(1 + np.exp(log_b-log_a))
    return v


class LDA :
# find the optimizing values of the vatiational parameters 
# alpha, beta : hyper-parameters
    
    def __init__(self,log_file,num_topic,dictionary,doc_term,itrMax = 50, itrEstep = 50, itrMstep = 50, stop = 1e-3):
        #self.alpha = np.ones(self.num_topic)
        #alpha = np.random.rand(num_topic)
        
        self.num_topic = num_topic
        self.dictionary = dictionary
        self.itrMax = itrMax 
        self.itrEstep = itrEstep
        self.itrMstep = itrMstep
        self.stop = 1e-3
        self.doc_term = doc_term

        self.corpus = []
        for i in range(self.doc_term.shape[0]) : 
            w_0 = np.where(self.doc_term[i,:] > 0 )
            w_1 = self.doc_term[i,w_0]
            words =  np.vstack((w_0,w_1)).T
            self.corpus.append(words)

        self.num_vocab = len(self.dictionary)      #number of vocabulary
        self.num_corpus = len(self.corpus)                   # number of document
        
        #self.alpha = np.ones(self.num_topic)
        self.alpha = np.array(range(self.num_topic)) + 1.0
        self.beta = np.zeros((self.num_topic,self.num_vocab))
        self.beta[:] = 1.0/self.num_vocab
        self.phi_doc = []  # list of size M
        self.gamma_doc = []  # list of size M 
    
    def E_step(self,words) : 
        num_word = words.shape[0]
        # initialization
        phi = np.zeros((num_word,self.num_topic))  # N*K
        phi[:] = 1.0/self.num_topic  
        gamma = np.zeros(self.num_topic) 
        gamma = self.alpha + num_word/self.num_topic

        converged_phi = 1.0
        converged_gamma = 1.0

        itr = 0
        while(itr <= self.itrEstep and (converged_phi > self.stop or converged_gamma > self.stop )) : 
            itr = itr + 1; 
            phi_new = np.zeros((num_word,self.num_topic))
            phisum = np.zeros((num_word))
            for n in range(0,num_word) :
                for i in range(0,self.num_topic) :
                    w_n = words[n,0]
                    # overflow because of exp 
                    #phi_new[n,i] = self.beta[i,w_n]  * np.exp(special.digamma(gamma[i])  -  special.digamma(np.sum(gamma))) 
                    if self.beta[i,w_n] == 0 : 
                        self.beta[i,w_n] = np.exp(-100)
                    phi_new[n,i] = np.log(self.beta[i,w_n]) + special.digamma(gamma[i])  -  special.digamma(np.sum(gamma))

                    if(i==0):
                        phisum[n] = phi_new[n,i]
                    else : 
                        phisum[n] = log_sum(phisum[n],phi_new[n,i])

                    if (np.isinf(phi_new[n,i])) : 
                        print "phi_new is inf", np.log(self.beta[i,w_n]),special.digamma(gamma[i]) - special.digamma(np.sum(gamma))

            #phi_new2 = phi_new / np.sum(phi_new, axis = 1)[:,None]

            phi_new2  = np.exp(phi_new - phisum[:,None])

            if (np.isnan(phi_new2).any()) :
                print phi_new, np.sum(phi_new, axis = 1)
            phi_new = phi_new2

            gamma_new = self.alpha + np.sum(phi_new, axis = 0)

            converged_phi = np.sum(np.abs(phi_new-phi))
            converged_gamma = np.sum(np.abs(gamma_new-gamma))

            phi = phi_new
            gamma = gamma_new
            #ll_new = self.log_likelihood([words],self.alpha,self.beta,[phi_new],[gamma_new])
            #print "E step : ", itr, ll_new
        return [phi, gamma]
        
            
    def M_step(self) :
        self.beta = np.zeros((self.num_topic,self.num_vocab))  #K*V      
        for m in range(self.num_corpus) :
            for n in range(self.corpus[m].shape[0]) :
                for i in range(self.num_topic) :
                    j = self.corpus[m][n,0]
                    self.beta[i,j] = self.beta[i,j] + self.phi_doc[m][n,i]  * self.corpus[m][n,1]
        self.beta = self.beta / np.sum(self.beta,axis = 1)[:,None]

        itr = 0
        ll_new = self.log_likelihood(self.corpus,self.alpha,self.beta,self.phi_doc,self.gamma_doc)
        converged = 1.0

        while(itr <= self.itrMstep and converged > self.stop) : 

            itr = itr + 1
            ll_old = ll_new 

            if(np.isnan(self.alpha).any()) : 
                self.alpha = self.alpha / 10.0

            g = np.zeros(self.num_topic)
            g = self.num_corpus * (special.digamma(np.sum(self.alpha)) - special.digamma(self.alpha))  #gradient 
            for d in range(0,self.num_corpus) : 
                g = g + special.digamma(self.gamma_doc[d]) - special.digamma(np.sum(self.gamma_doc[d]))

            h =  - self.num_corpus * special.polygamma(1,self.alpha) # vector along the diagonal of hessien
            z =  special.polygamma(1,np.sum(self.alpha))   # constant
            c = np.sum(g/h)/(1.0/z + np.sum(1.0/h))
            Hg = (g-c)/h

            self.alpha = self.alpha -  Hg
            ll_new = self.log_likelihood(self.corpus,self.alpha,self.beta,self.phi_doc,self.gamma_doc)

            converged = np.abs(ll_new-ll_old)
            #print "M step : ", itr, ll_new, np.sqrt(np.sum(Hg**2))

        return[self.beta,self.alpha]

            
    def log_likelihood(self,corpus,alpha,beta,phi_doc,gamma_doc) : 
        l = 0
        M = len(phi_doc) # number of document
        for m in range(0,M) : 
            gamma = gamma_doc[m]
            phi = phi_doc[m]
            words = corpus[m]
            len_word = words.shape[0]
            term1 = special.gammaln(np.sum(alpha)) - np.sum(special.gammaln(alpha)) \
                    + np.sum((alpha-1)*(special.digamma(gamma)- special.digamma(np.sum(gamma))))
            term2 = np.sum(phi * (special.digamma(gamma)- special.digamma(np.sum(gamma)))) 

            l += term1 + term2

            term3 = 0
            for n in range(0,len_word) :
                w_n = words[n,0]
                term3 += np.sum(phi[n,:] * np.log(beta[:,w_n]))
            l += term3

            term4 = - special.gammaln(np.sum(gamma)) + np.sum(special.gammaln(gamma)) \
                  - np.sum((gamma-1)*(special.digamma(gamma)- special.digamma(np.sum(gamma))))

            term5 = 0  #term5 = - np.sum(phi * np.log(phi))
            for n in range(0,len_word) :
                for i in range(0,num_topic):
                    if(phi[n,i] > 0 ) :
                        term5 += phi[n,i] * np.log(phi[n,i])

            l += term4 + term5

            if(np.isnan(term4) or np.isinf(term4)) : 
                print "term4 have nan!!!", gamma 

            if(np.isnan(term5) or np.isinf(term5)) : 
                print "term5 have nan!!!", phi 

        return l


    def train(self) : 
        
        converged = 1.0
        itr = 0
        ll_new = 0.0
        
        while(itr <= self.itrMax and converged > 1e-3 ) :
            itr = itr + 1
            ll_old = ll_new
            self.phi_doc = []  # list of size M
            self.gamma_doc = []  # list of size M 
            for m in range(0,len(self.corpus)) : # E step  : 
                [phi,gamma] = self.E_step(self.corpus[m])
                self.phi_doc.append(phi)
                self.gamma_doc.append(gamma)   

            ## M step
            [beta,alpha] = self.M_step()
            ll_new = self.log_likelihood(self.corpus,self.alpha,self.beta,self.phi_doc,self.gamma_doc)
            converged = np.abs(ll_new - ll_old)

            print " EM step iteration " , itr , ll_new
            log_string = " EM step iteration " + str(itr) +" : "+ str(ll_new) + "\n"
            log_file.write(log_string)

        return [ll_new, self.alpha, self.beta,self.phi_doc,self.gamma_doc]
  


In [7]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

print(ldamodel.print_topics(num_topics=2, num_words=3))


[(0, u'0.010*"s" + 0.009*"t" + 0.008*"b"'), (1, u'0.013*"s" + 0.008*"israel" + 0.008*"will"')]
359.13420537 4.60016185274
4.60016185274 4.60016185274


In [ ]:
doc_set = [
    'weather: warm, cold, freezing, hot, windy, warm',
    'weather: dry, windy, moist, cold, etc',
    'freezing means dry and windy',
    'sports game, be it basketball, hockey or soccer, I feel better, hockey',
    'sports can be soothing. hockey, but I like soccer and basketball'
]

log_file = open("error_loss.txt", "w")
    
n_features = 1000
n_components = 10
n_top_words = 20
n_samples = 2000

dataset = fetch_20newsgroups(shuffle=True, random_state=1,remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
doc_set = data_samples[:n_samples]

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,stop_words='english')
tf = tf_vectorizer.fit_transform(doc_set)
dictionary =  tf_vectorizer.vocabulary_
doc_term = tf.toarray()

print len(dictionary)
print doc_term.shape

id2token = {}
for token in dictionary.keys() :
    id2token[dictionary[token]] = token

num_topic = 10
num_w = 20

model = LDA(log_file,num_topic,dictionary,doc_term,itrMax=100)
[ll,alpha, beta,phi_doc,gamma_doc] = model.train()

weights = -np.sort(-beta,axis = 1)[:,:num_w]
res = np.argsort(-beta,axis = 1)[:,:num_w]

print weights

for i in range(0,res.shape[0]) :
    topic = {}
    for j in range(0,res.shape[1]) : 
        topic[id2token[res[i,j]]]= round(weights[i,j],2)  
    print topic.keys()
    string ="topic " + str(i) + " : " +  ";".join(str(value) for value in topic) + "\n"
    log_file.write(string)
    
    
# probability of each tropic of each document
for i in range(len(gamma_doc)) :
    theta = np.exp(special.digamma(gamma_doc[i]) - special.digamma(np.sum(gamma_doc[i])))
    string ="doc " + str(i) + " : " + ";".join(str(value) for value in theta) + "\n"
    log_file.write(string)
    #print theta

log_file.close()


1000
(2000, 1000)
 EM step iteration  1 -566977.909174
 EM step iteration  2 -566461.063043
 EM step iteration  3 -566116.269805
 EM step iteration  4 -565872.35171
 EM step iteration  5 -565691.985544
 EM step iteration  6 -565553.791208
 EM step iteration  7 -565444.76983
 EM step iteration  8 -565356.761547
 EM step iteration  9 -565284.361392
 EM step iteration  10 -565223.816992
 EM step iteration  11 -565172.511024
 EM step iteration  12 -565128.528414
 EM step iteration  13 -565090.450366
 EM step iteration  14 -565057.228746
 EM step iteration  15 -565027.997367
 EM step iteration  16 -565002.112739
 EM step iteration  17 -564979.046699
